<a href="https://colab.research.google.com/github/rendihardi/OlahData-AspectBased-SentimentAnalysis/blob/main/Pemodelan_BERTopic.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [14]:
import pandas as pd
import numpy as np

df = pd.read_csv('/content/olahdatafix.csv')

In [15]:
df

,at,userName,content,score,content_clean,content_casefolded,content_normalized,content_stopword,content_tokens,content_stemmed
0,12/30/2024 23:55,Pengguna Google,LEBIH simpel BNI MOBILE. MAKSUDNYA APA YAH nas...,1,LEBIH simpel BNI MOBILE MAKSUDNYA APA YAH nasa...,lebih simpel bni mobile maksudnya apa yah nasa...,lebih simpel bni mobile maksudnya apa ya nasab...,lebih simpel maksudnya nasabah suruh migrasi f...,"['lebih', 'simpel', 'maksudnya', 'nasabah', 's...","['lebih', 'simpel', 'maksud', 'nasabah', 'suru..."
1,12/30/2024 23:15,Pengguna Google,"daftarin ortu,verifikasi wajah gagal terus, ja...",1,daftarin ortu verifikasi wajah gagal terus jad...,daftarin ortu verifikasi wajah gagal terus jad...,daftar orang tua verifikasi wajah gagal terus ...,daftar orang tua verifikasi wajah gagal limit ...,"['daftar', 'orang', 'tua', 'verifikasi', 'waja...","['daftar', 'orang', 'tua', 'verifikasi', 'waja..."
2,12/30/2024 22:45,Pengguna Google,Lebih simpel dari aplikasi BNI sebelumnya,5,Lebih simpel dari aplikasi BNI sebelumnya,lebih simpel dari aplikasi bni sebelumnya,lebih simpel dari aplikasi bni sebelumnya,lebih simpel,"['lebih', 'simpel']","['lebih', 'simpel']"
3,12/30/2024 18:49,Pengguna Google,"Wah, notifikasi terkait konfirmasi transaksi l...",2,Wah notifikasi terkait konfirmasi transaksi la...,wah notifikasi terkait konfirmasi transaksi la...,wah notifikasi terkait konfirmasi transaksi la...,notifikasi terkait konfirmasi transaksi lama k...,"['notifikasi', 'terkait', 'konfirmasi', 'trans...","['notifikasi', 'kait', 'konfirmasi', 'transaks..."
4,12/30/2024 17:15,Pengguna Google,Mengapa memakai baras minimum jika ingin trans...,5,Mengapa memakai baras minimum jika ingin trans...,mengapa memakai baras minimum jika ingin trans...,mengapa memakai baras minimum jika ingin trans...,memakai baras minimum transaksi tidak memakai ...,"['memakai', 'baras', 'minimum', 'transaksi', '...","['pakai', 'baras', 'minimum', 'transaksi', 'ti..."
...,...,...,...,...,...,...,...,...,...,...
18214,7/31/2024 6:58,Irfann_,Kalo bayar bayar sekarang pake wondr jadi mudah,5,Kalo bayar bayar sekarang pake wondr jadi mudah,kalo bayar bayar sekarang pake wondr jadi mudah,kalau bayar bayar sekarang pakai wondr jadi mudah,bayar bayar pakai mudah,"['bayar', 'bayar', 'pakai', 'mudah']","['bayar', 'bayar', 'pakai', 'mudah']"
18215,7/31/2024 5:14,Batalkan Tulis Ulasan Kirim,Aplikasi baru ni tidak ada menu pilihan mutasi...,2,Aplikasi baru ni tidak ada menu pilihan mutasi...,aplikasi baru ni tidak ada menu pilihan mutasi...,aplikasi baru saja ini tidak ada menu pilihan ...,tidak menu pilihan histori transaksi rekening ...,"['tidak', 'menu', 'pilihan', 'histori', 'trans...","['tidak', 'menu', 'pilih', 'histori', 'transak..."
18216,7/31/2024 3:45,nurinpdewi,Thank’s BNI,5,Thanks BNI,thanks bni,thanks bni,thanks,['thanks'],['thanks']
18217,7/31/2024 3:10,Tiwiw17,Gabisa verifikasi wajah udh berkali-kali,1,Gabisa verifikasi wajah udh berkali kali,gabisa verifikasi wajah udh berkali kali,tidak bisa verifikasi wajah sudah berkali kali,tidak verifikasi wajah berkali kali,"['tidak', 'verifikasi', 'wajah', 'berkali', 'k...","['tidak', 'verifikasi', 'wajah', 'kali', 'kali']"


**Pemodelan BERTopic**

In [ ]:
!pip install umap-learn
!pip install hdbscan
!pip install bertopic

import umap
import pandas as pd
from sentence_transformers import SentenceTransformer
from hdbscan import HDBSCAN
from bertopic import BERTopic
from bertopic.vectorizers import ClassTfidfTransformer
from sklearn.feature_extraction.text import CountVectorizer

# Seed topics
seed_topics = [
    ["transaksi", "pembayaran", "transfer", "saldo"],
    ["akun", "login", "registrasi", "verifikasi", "daftar", "aktivasi"],
    ["tampilan", "desain", "uiux", "warna", "layout",],
    ["kecepatan", "loading", "responsif", "lemot", "lag", "performa"],
    ["customer service", "cs", "bantuan", "komplain", "respon"],
    ["error", "bug", "crash", "gagal", "kendala"],
    ["fitur", "inovasi", "update", "pengembangan"],
]


# Load embedding model
sentence_model = SentenceTransformer("paraphrase-multilingual-MiniLM-L12-v2")
# UMAP model
umap_model = umap.UMAP(n_neighbors=15, n_components=6, min_dist=0.3, metric='cosine')
# HDBSCAN model
hdbscan_model = HDBSCAN(
    min_cluster_size=15,
    metric='euclidean',
    cluster_selection_method='leaf',
    prediction_data=True
)
# CountVectorizer
vectorizer_model = CountVectorizer()
# Class TF-IDF Transformer
ctfidf_model = ClassTfidfTransformer(bm25_weighting=True, reduce_frequent_words=True)

# BERTopic Model
model = BERTopic(
    embedding_model=sentence_model,
    umap_model=umap_model,
    hdbscan_model=hdbscan_model,
    vectorizer_model=vectorizer_model,
    ctfidf_model=ctfidf_model,
    seed_topic_list=seed_topics,
    language='multilingual',
    nr_topics=4,
    min_topic_size=800,
    calculate_probabilities=True,
    verbose=True
)

# Pastikan df sudah didefinisikan
if 'df' in globals() and 'content_stopword' in df.columns:
    topics, probabilities = model.fit_transform(df['content_stopword'])
else:
    raise ValueError("DataFrame 'df' belum didefinisikan atau kolom 'content_stopword' tidak ditemukan.")


2025-02-21 12:37:20,620 - BERTopic - Embedding - Transforming documents to embeddings.


Batches:   0%|          | 0/570 [00:00<?, ?it/s]

2025-02-21 12:41:39,174 - BERTopic - Embedding - Completed ✓
2025-02-21 12:41:39,176 - BERTopic - Guided - Find embeddings highly related to seeded topics.


Batches:   0%|          | 0/1 [00:00<?, ?it/s]

2025-02-21 12:41:39,424 - BERTopic - Guided - Completed ✓
2025-02-21 12:41:39,425 - BERTopic - Dimensionality - Fitting the dimensionality reduction algorithm
2025-02-21 12:42:01,581 - BERTopic - Dimensionality - Completed ✓
2025-02-21 12:42:01,586 - BERTopic - Cluster - Start clustering the reduced embeddings
2025-02-21 12:42:30,848 - BERTopic - Cluster - Completed ✓
2025-02-21 12:42:30,849 - BERTopic - Representation - Extracting topics from clusters using representation models.
2025-02-21 12:42:31,177 - BERTopic - Representation - Completed ✓
2025-02-21 12:42:31,178 - BERTopic - Topic reduction - Reducing number of topics
2025-02-21 12:42:31,403 - BERTopic - Topic reduction - Reduced number of topics from 184 to 4


**Representasi Topic**

In [ ]:
topic_info = model.get_topic_info()
topic_info


,Topic,Count,Name,Representation,Representative_Docs
0,-1,10230,-1_transaksi_fitur_tidak_pakai,"[transaksi, fitur, tidak, pakai, login, nggak,...",[keren sih sangat memudahkan bertransaksi ngga...
1,0,4912,0_mudah_tampilan_cepat_sangat,"[mudah, tampilan, cepat, sangat, simpel, trans...","[mudah praktis tampilan sangat menarik, tampil..."
2,1,1768,1_verifikasi_login_email_error,"[verifikasi, login, email, error, gagal, updat...",[kode otp verifikasi email lama dikirim masuk ...
3,2,1309,2_mobile_transfer_tunai_tarik,"[mobile, transfer, tunai, tarik, kartu, saldo,...",[bagus sih belum fitur mobile tunai transaksi ...


In [ ]:
model.get_topic(0)

[('mudah', 0.5904114852763012),
 ('tampilan', 0.5806176706068079),
 ('cepat', 0.5355401850958286),
 ('sangat', 0.5052076491304482),
 ('simpel', 0.4971537622534343),
 ('transaksi', 0.47497167501518894),
 ('mantap', 0.4704782594236069),
 ('menarik', 0.464871009819693),
 ('bagus', 0.46368540360290667),
 ('keren', 0.46072731682428636)]

In [ ]:
model.get_topic(1)

[('verifikasi', 0.8065197824113529),
 ('login', 0.7072272119246096),
 ('email', 0.6382380507544781),
 ('error', 0.6154411723895635),
 ('gagal', 0.5677338973396876),
 ('update', 0.5439501684375535),
 ('wajah', 0.5381834921021683),
 ('otp', 0.49940155928461283),
 ('kode', 0.4869003618606809),
 ('susah', 0.45708097321540947)]

In [ ]:
model.get_topic(2)

[('mobile', 0.6489180392190066),
 ('transfer', 0.621943786188827),
 ('tunai', 0.5957571774421296),
 ('tarik', 0.5471953284114953),
 ('kartu', 0.5344397634875032),
 ('saldo', 0.531263873270897),
 ('tanpa', 0.5202421136333084),
 ('fitur', 0.49336700995458627),
 ('banking', 0.49069513107481766),
 ('transaksi', 0.46746209897476615)]

**Evaluasi Coherence**

In [ ]:
from bertopic import BERTopic
from gensim.models.coherencemodel import CoherenceModel
from gensim.corpora.dictionary import Dictionary

# Dapatkan kata-kata untuk setiap topik
topics = model.get_topics()
topic_words = [[word for word, _ in topic] for topic in topics.values()]

# Tokenisasi teks
tokens = [doc.split() for doc in df['content_stopword']]

# Buat dictionary dan corpus
dictionary = Dictionary(tokens)
corpus = [dictionary.doc2bow(text) for text in tokens]

# Hitung coherence score
coherence_model = CoherenceModel(topics=topic_words, texts=tokens, dictionary=dictionary, coherence='c_v')
coherence_score = coherence_model.get_coherence()

print(f"Coherence Score: {coherence_score:.4f}")


Coherence Score: 0.5688


**Evaluasi Diversity**

In [ ]:
# Tentukan jumlah kata teratas yang akan dievaluasi dari tiap topik
top_n_words = 10

# Dapatkan informasi topik, lalu ambil ID topik yang valid (hilangkan topik -1 jika ada)
topic_info = model.get_topic_info()
valid_topics = topic_info[topic_info.Topic != -1]['Topic'].tolist()

# Ambil top_n_words untuk tiap topik
topic_words = []
for topic_id in valid_topics:
    # model.get_topic(topic_id) mengembalikan list tuple (kata, skor)
    topic = model.get_topic(topic_id)
    # Ambil hanya kata-katanya dari top_n_words pertama
    words = [word for word, score in topic[:top_n_words]]
    topic_words.append(words)

# Gabungkan semua kata dari tiap topik ke dalam satu set untuk mendapatkan kata unik
unique_words = set(word for words in topic_words for word in words)

# Total kata yang diharapkan adalah top_n_words * jumlah topik yang dianalisis
total_words = top_n_words * len(topic_words)

# Hitung Topic Diversity Score
topic_diversity_score = len(unique_words) / total_words
print(f"Topic Diversity Score: {topic_diversity_score:.4f}")

Topic Diversity Score: 0.9667
